In [ ]:
import pandas as pd

In [ ]:
df = pd.concat([pd.read_csv('hf-endpoint/bench-hf-endpoint.csv').assign(platform='Inference Endpoint (A10G)').assign(options='-'),
                pd.read_csv('tgi/bench-default.csv').assign(platform='TGI').assign(options='-'), 
                pd.read_csv('tgi/bench-quantize-bb.csv').assign(platform='TGI').assign(options='quantized w/ bitsandbytes'),
                pd.read_csv('tgi/bench-quantize-gptq.csv').assign(platform='TGI').assign(options='quantized w/ GPTQ'),
                pd.read_csv('exllama/bench-exllama.csv').assign(platform='text-generation-webui').assign(options='exllama'),
                pd.read_csv('exllama/bench-ggmlcpp.csv').assign(platform='text-generation-webui').assign(options='ggml'),
                pd.read_csv('vllm/bench-vllm.csv').assign(platform='vllm').assign(options='-'),
                pd.read_csv('vllm/modal-examples/bench-vllm.csv').assign(platform='vllm on A100').assign(options='-')]
              )

In [ ]:
df.head()

,tok_count,time,question,answer,note,platform,options
0,202,6.6,Write a Rust function that performs binary exp...,"\n\n## Examples\n\n```rust\nexponentiate(2, 3)...",hf-endpoint,Inference Endpoint (A10G),-
1,202,6.9,What are the differences between Javascript an...,\nPython is a high-level programming language ...,hf-endpoint,Inference Endpoint (A10G),-
2,202,6.6,Write a story in the style of James Joyce abou...,\nWrite a story in the style of James Joyce ab...,hf-endpoint,Inference Endpoint (A10G),-
3,202,6.6,Who does Harry turn into a balloon?,\nWho does Harry turn into a balloon? Harry Po...,hf-endpoint,Inference Endpoint (A10G),-
4,202,6.6,Write a tale about a time-traveling historian ...,\nWrite a tale about a time-traveling historia...,hf-endpoint,Inference Endpoint (A10G),-


In [ ]:
df['tok/sec'] = df['tok_count'] / df['time']

In [ ]:
pd.set_option('display.precision', 1)
df.groupby(['platform', 'options']).mean('time')[['tok/sec']]

tok/sec
platform                  options                           
Inference Endpoint (A10G) -                             30.4
TGI                       -                             21.1
                          quantized w/ GPTQ             23.6
                          quantized w/ bitsandbytes      1.9
text-generation-webui     exllama                       30.8
                          ggml                           9.9
vllm                      -                             46.3
vllm on A100              -                             40.9

Why is quantization slower for `bits and bytes`?  Someone else [mentioned it here](https://github.com/huggingface/text-generation-inference/issues/309) and says that it is well known that it is slow!  